Импорт библиотек

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

Импорт листов

In [25]:
description = pd.read_excel('data/socio.xls', sheet_name='Description', skiprows=4, index_col=0) #описание исходных данных

#листы
M1 = pd.read_excel('data/socio.xls', sheet_name='M1')
M2 = pd.read_excel('data/socio.xls', sheet_name='M2')
M3 = pd.read_excel('data/socio.xls', sheet_name='M3')
M4 = pd.read_excel('data/socio.xls', sheet_name='M4')
M5 = pd.read_excel('data/socio.xls', sheet_name='M5')
M6 = pd.read_excel('data/socio.xls', sheet_name='M6')
M7 = pd.read_excel('data/socio.xls', sheet_name='M7')
M8 = pd.read_excel('data/socio.xls', sheet_name='M8')
M9 = pd.read_excel('data/socio.xls', sheet_name='M9')

In [26]:
description

,Вопрос
Лист,
M1,1.С кем бы Вы хотели сидеть за одной партой?
M2,2.С кем бы Вы НЕ хотели сидеть за одной партой?
M3,3.Кого бы Вы хотели пригласить на свой День Ро...
M4,4.Кого бы Вы НЕ хотели пригласить на свой День...
M5,5.Кого бы Вы назначили старостой группы?
M6,6.Кого бы Вы НЕ назначили старостой группы?
M7,7.С кем из своих одногруппников Вы бы хотели п...
M8,8.С кем из своих одногруппников Вы бы НЕ хотел...
M9,9.Кого из одногруппников Вы считаете своим дру...
